In [ ]:
import os
import subprocess
import sys
from typing import List
from pyspark.sql import SparkSession
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory
jars_home = '/workspace/delta-jars'

# Required core JARs
jars_list = [
    # Delta Lake
    f"{jars_home}/delta-spark_2.12-3.2.1.jar",
    f"{jars_home}/delta-storage-3.2.1.jar",
    # AWS
    f"{jars_home}/hadoop-aws-3.3.2.jar",
    f"{jars_home}/aws-java-sdk-bundle-1.12.261.jar",
    # Kyuubi
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-spark-sql-engine_2.12-1.7.3.jar",
    f"{jars_home}/kyuubi/externals/engines/spark/kyuubi-common_2.12-1.7.3.jar"
]

# Convert to comma-separated string
jars = ",".join(jars_list)

# Create SparkSession using the builder pattern
builder = (SparkSession.builder
           .appName("DeltaExample")
           .master("local[*]")
           # Add debug configurations
           .config("spark.hadoop.fs.s3a.connection.maximum", "1")
           .config("spark.hadoop.fs.s3a.attempts.maximum", "1")
           .config("spark.hadoop.fs.s3a.connection.timeout", "5000")
           .config("spark.hadoop.fs.s3a.impl.disable.cache", "true")
           .config("spark.hadoop.fs.s3a.debug.detailed.exceptions", "true")
           # Add jars directly
           .config("spark.jars", jars)
           .config("spark.driver.extraClassPath", jars)
           .config("spark.executor.extraClassPath", jars)
           # Delta Lake configurations
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
           # S3/MinIO configurations
           .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
           .config("spark.hadoop.fs.s3a.path.style.access", "true")
           .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
           .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
           .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
           # Additional Delta Lake configurations
           .config("spark.delta.logStore.class", "io.delta.storage.S3SingleDriverLogStore")
           .config("spark.hadoop.fs.s3a.fast.upload", "true")
           .config("spark.hadoop.fs.s3a.multipart.size", "104857600")
           .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse"))

# Stop any existing session
if 'spark' in locals():
    spark.stop()


# Create the session
spark = builder.enableHiveSupport().getOrCreate()

# Initialize Delta Lake settings
# spark.sql("SET spark.databricks.delta.formatCheck.enabled=false")

# Access the SparkContext
# sc = spark.sparkContext

# Set the log level to INFO
# sc.setLogLevel("DEBUG")

# Test DataFrame
data = [(1, "John"), (2, "Jane")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()

# First verify the S3 connection by listing the bucket
try:
    # Try to write to a simple parquet file first to test S3 connection
    print("Testing S3 connection with parquet write...")
    df.write.format("parquet").mode("overwrite").save("s3a://wba/test.parquet")
    print("S3 connection successful")

    print("Attempting to write Delta table...")
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("delta.compatibility.symlinkFormatManifest.enabled", "false") \
        .save("s3a://wba/example-table")
    print("Successfully wrote Delta table")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTrying local filesystem instead...")
    try:
        local_path = "/tmp/test-delta-table"
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(local_path)
        print(f"Successfully wrote to {local_path}")
    except Exception as local_e:
        print(f"Error writing to local filesystem: {str(local_e)}")

25/02/21 15:52:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/21 15:52:03 WARN DependencyUtils: Local jar /workspace/delta-jars/kyuubi/externals/engines/spark/kyuubi-spark-sql-engine_2.12-1.7.3.jar does not exist, skipping.
25/02/21 15:52:03 WARN DependencyUtils: Local jar /workspace/delta-jars/kyuubi/externals/engines/spark/kyuubi-common_2.12-1.7.3.jar does not exist, skipping.
25/02/21 15:52:03 INFO SparkContext: Running Spark version 3.5.4
25/02/21 15:52:03 INFO SparkContext: OS info Linux, 5.15.167.4-microsoft-standard-WSL2, amd64
25/02/21 15:52:03 INFO SparkContext: Java version 17.0.14
25/02/21 15:52:03 INFO ResourceUtils: ==============================================================
25/02/21 15:52:03 INFO ResourceUtils: No custom resources configured for spark.driver.
25/02/21 15:52:03 INFO ResourceUtils: ==============================================================
25/02/21 15:52:03 INFO Sp

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Jane|
+---+----+

Testing S3 connection with parquet write...


25/02/21 15:52:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/02/21 15:52:13 INFO MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
25/02/21 15:52:13 INFO MetricsSystemImpl: s3a-file-system metrics system started
25/02/21 15:52:14 INFO BlockManagerInfo: Removed broadcast_2_piece0 on 9d43eaa40a2f:45463 in memory (size: 6.6 KiB, free: 434.4 MiB)
25/02/21 15:52:15 INFO ParquetUtils: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
25/02/21 15:52:15 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/02/21 15:52:15 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/02/21 15:52:15 INFO SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
25/02/21 15:52:15 INFO File

S3 connection successful
Attempting to write Delta table...


25/02/21 15:52:16 INFO DeltaLog: Loading version 9.
25/02/21 15:52:17 INFO DeltaLogFileIndex: Created DeltaLogFileIndex(JSON, numFilesInSegment: 10, totalFileSize: 14255)
25/02/21 15:52:17 INFO DataSourceStrategy: Pruning directories with: 
25/02/21 15:52:17 INFO FileSourceStrategy: Pushed Filters: 
25/02/21 15:52:17 INFO FileSourceStrategy: Post-Scan Filters: ((isnotnull(protocol#30.minReaderVersion) OR isnotnull(metaData#29.id)) OR (isnotnull(commitInfo#31.inCommitTimestamp) AND (version#32L = 9)))
25/02/21 15:52:17 INFO CodeGenerator: Code generated in 87.534333 ms
25/02/21 15:52:17 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 217.9 KiB, free 433.9 MiB)
25/02/21 15:52:17 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 36.7 KiB, free 433.9 MiB)
25/02/21 15:52:17 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on 9d43eaa40a2f:45463 (size: 36.7 KiB, free: 434.3 MiB)
25/02/21 15:52:17 INFO SparkContext: Cre

Successfully wrote Delta table


25/02/21 15:52:28 INFO DeltaLog: Deleted 0 log files older than 22 Jan 2025 00:00:00 GMT
